This notebook uses different techniques to create models for classification of the images of letters in the dataset.

Firstly we import the libraries we'll need.

In [ ]:
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
from PIL import Image

import keras
import autokeras as ak
from keras.preprocessing.image import load_img
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation, BatchNormalization
from keras.layers import Reshape, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.utils import np_utils
from keras.regularizers import l1_l2
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

%matplotlib inline
%env JOBLIB_TEMP_FOLDER=/tmp

Select which data to use.

In [ ]:
dataset = 'notMNIST_small'
DATA_PATH = '../input/' + dataset + '/' + dataset

Check some data from the training dataset

In [ ]:
max_images = 100
grid_width = 10
grid_height = int(max_images / grid_width)
fig, axs = plt.subplots(grid_height, grid_width, figsize=(grid_width, grid_height))
classes = os.listdir(DATA_PATH)
for j, cls in enumerate(classes):
    figs = os.listdir(DATA_PATH + '/' + cls)
    for i, fig in enumerate(figs[:grid_width]):
        ax = axs[j, i]
        ax.imshow(np.array(load_img(DATA_PATH + '/' + cls + '/' + fig)))
        ax.set_yticklabels([])
        ax.set_xticklabels([])

Load images and make them ready for fitting a model.

In [ ]:
X = []
labels = []
# for each folder (holding a different set of letters)
for directory in os.listdir(DATA_PATH):
    # for each image
    for image in os.listdir(DATA_PATH + '/' + directory):
        # open image and load array data
        try:
            file_path = DATA_PATH + '/' + directory + '/' + image
            img = Image.open(file_path)
            img.load()
            img_data = np.asarray(img, dtype=np.int16)
            # add image to dataset
            X.append(img_data)
            # add label to labels
            labels.append(directory)
        except:
            None # do nothing if couldn't load file
N = len(X) # number of images
img_size = len(X[0]) # width of image
X = np.asarray(X).reshape(N, img_size, img_size,1) # add our single channel for processing purposes
labels_cat = to_categorical(list(map(lambda x: ord(x)-ord('A'), labels)), 10) # convert to one-hot
labels = np.asarray(list(map(lambda x: ord(x)-ord('A'), labels)))

Check balance of classes.

In [ ]:
cls_s = np.sum(labels,axis=0)

fig, ax = plt.subplots()
plt.bar(np.arange(10), cls_s)
plt.ylabel('No of pics')
plt.xticks(np.arange(10), np.sort(classes))
plt.title('Checking balance for data set..')
plt.show()

Divide data into train/test datasets.

In [ ]:
from sklearn.cross_validation import train_test_split
X_train,X_valid,y_train,y_valid=train_test_split(X,labels,test_size=0.2)
X_train_cat,X_valid_cat,y_train_cat,y_valid_cat=train_test_split(X,labels_cat,test_size=0.2)

print('Training:', X_train.shape, y_train.shape)
print('Validation:', X_valid.shape, y_valid.shape)

Let's shuffle the data for a better conditioning of the sets. (useless?)

In [ ]:
# def randomize(dataset, labels):
#   permutation = np.random.permutation(labels.shape[0])
#   shuffled_dataset = dataset[permutation,:,:]
#   shuffled_labels = labels[permutation]
#   return shuffled_dataset, shuffled_labels
#   
# X_train, y_train = randomize(X_train, y_train)
# X_test, y_test = randomize(X_test, y_test)
# X_train_cat, y_train_cat = randomize(X_train_cat, y_train_cat)
# X_test_cat, y_test_cat = randomize(X_test_cat, y_test_cat)

Sanity check of the final dataset.

In [ ]:
fig, axs = plt.subplots(grid_height, grid_width, figsize=(grid_width, grid_height))
for j in range(max_images):
    ax = axs[int(j/grid_width), j%grid_width]
    ax.imshow(X_train[j,:,:,0])
    ax.set_yticklabels([])
    ax.set_xticklabels([])

Prepare the data.

In [ ]:
samples, width, height = np.squeeze(X_train).shape
X_train_lr = np.reshape(X_train,(samples,width*height))
y_train_lr = y_train

samples, width, height = np.squeeze(X_valid).shape
X_valid_lr = np.reshape(X_valid,(samples,width*height))
y_valid_lr = y_valid

Let's see what AutoKeras produces:

In [ ]:
clf = ak.ImageClassifier(verbose=1,augment=1)
clf.fit(X_train_cat, y_train_cat)
clf.final_fit(X_train_cat, y_train_cat, X_test_cat, y_test_cat, retrain=True)
results = clf.predict(X_test_cat)

score = clf.evaluate(X_test_cat, y_test_cat, verbose=0)

print('Test score:', score[0])
print('Test accuracy:', score[1])

So, the architecture found by this autoML algorithm is:

In [ ]:
... to test ...